In [21]:
# From Siraj Raval
# https://www.youtube.com/watch?v=9zhrxE5PQgY

In [1]:
import numpy as np

In [38]:
class LSTM:
    def __init__(self, xs, ys, rl, lr):
        self.x = np.zeros(xs + ys)
        self.xs = xs + ys
        #output 
        self.y = np.zeros(ys)
        #output size
        self.ys = ys
        #cell state intialized as size of prediction
        self.cs = np.zeros(ys)
        #how often to perform recurrence
        self.rl = rl
        #balance the rate of training (learning rate)
        self.lr = lr
        #init weight matrices for our gates
        #forget gate
        self.f = np.random.random((ys, xs+ys))
        #input gate
        self.i = np.random.random((ys, xs+ys))
        #cell state
        self.c = np.random.random((ys, xs+ys))
        #output gate
        self.o = np.random.random((ys, xs+ys))
        #forget gate gradient
        self.Gf = np.zeros_like(self.f)
        #input gate gradient
        self.Gi = np.zeros_like(self.i)
        #cell state gradient
        self.Gc = np.zeros_like(self.c)
        #output gate gradient
        self.Go = np.zeros_like(self.o)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(x))
    
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def tangent(self, x):
        return np.tanh(x)
    
    def dtangent(self, x):
        return 1 - np.tanh(x) ** 2
    
    def forwardProp(self):
        f = self.sigmoid(np.dot(self.f, self.x))
        self.cs *= f
        i = self.sigmoid(np.dot(self.i, self.x))
        c = self.tangent(np.dot(self.c, self.x))
        self.cs += i * c
        o = self.sigmoid(np.dot(self.o, self.x))
        self.y = o * self.tangent(self.cs)
        return self.cs, self.y, f, i, c, o
    
    def backProp(self, e, pcs, f, i, c, o, dfcs, dfhs):
        #error = error + hidden state derivative. clip the value between -6 and 6.
        e = np.clip(e + dfhs, -6, 6)
        #multiply error by activated cell state to compute output derivative
        do = self.tangent(self.cs) * e
        #output update = (output deriv * activated output) * input
        ou = np.dot(np.atleast_2d(do * self.dtangent(o)).T, np.atleast_2d(self.x))
        #derivative of cell state = error * output * deriv of cell state + deriv cell
        dcs = np.clip(e * o * self.dtangent(self.cs) + dfcs, -6, 6)
        #deriv of cell = deriv cell state * input
        dc = dcs * i
        #cell update = deriv cell * activated cell * input
        cu = np.dot(np.atleast_2d(dc * self.dtangent(c)).T, np.atleast_2d(self.x))
        #deriv of input = deriv cell state * cell
        di = dcs * c
        #input update = (deriv input * activated input) * input
        iu = np.dot(np.atleast_2d(di * self.dsigmoid(i)).T, np.atleast_2d(self.x))
        #deriv forget = deriv cell state * all cell states
        df = dcs * pcs
        #forget update = (deriv forget * deriv forget) * input
        fu = np.dot(np.atleast_2d(df * self.dsigmoid(f)).T, np.atleast_2d(self.x))
        #deriv cell state = deriv cell state * forget
        dpcs = dcs * f
        #deriv hidden state = (deriv cell * cell) * output + deriv output * output * output deriv input * input * output + deriv forget
        #* forget * output
        dphs = np.dot(dc, self.c)[:self.ys] + np.dot(do, self.o)[:self.ys] + np.dot(di, self.i)[:self.ys] + np.dot(df, self.f)[:self.ys] 
        #return update gradinets for forget, input, cell, output, cell state, hidden state
        return fu, iu, cu, ou, dpcs, dphs
    
    def update(self, fu, iu, cu, ou):
        #update forget, input, cell, and output gradients
        self.Gf = 0.9 * self.Gf + 0.1 * fu**2 
        self.Gi = 0.9 * self.Gi + 0.1 * iu**2   
        self.Gc = 0.9 * self.Gc + 0.1 * cu**2   
        self.Go = 0.9 * self.Go + 0.1 * ou**2   
        
        #update our gates using our gradients
        self.f -= self.lr/np.sqrt(self.Gf + 1e-8) * fu
        self.i -= self.lr/np.sqrt(self.Gi + 1e-8) * iu
        self.c -= self.lr/np.sqrt(self.Gc + 1e-8) * cu
        self.o -= self.lr/np.sqrt(self.Go + 1e-8) * ou
        return

In [39]:
class RecurrentNeuralNetwork:
    # xs: seed word, ys: next word, rl: num words, eo: array of expected outputs, lr:learning rate
    def __init__(self, xs, ys, rl, eo, lr):
        # Initial input
        self.x = np.zeros(xs)
        # Input Size
        self.xs = xs
        # Expected Output (next word)
        self.y = np.zeros(ys)
        # Output Size
        self.ys = ys
        self.rl = rl
        self.lr = lr
        # Weight Matrix for interpreting results from LSTM cell
        self.w = np.random.random((ys,ys))
        # Matrix for RMS_prop
        self.G = np.zeros_like(self.w)
        # Inputs
        self.ia = np.zeros((rl+1, xs))
        # Cell states
        self.ca = np.zeros((rl+1, ys))
        # Outputs
        self.oa = np.zeros((rl+1, ys))
        # Hidden states
        self.ha = np.zeros((rl+1, ys))
        # Forget Gate
        self.af = np.zeros((rl+1, ys))
        # Input Gate
        self.ai = np.zeros((rl+1, ys))
        # Cell Gate
        self.ac = np.zeros((rl+1, ys))
        # Output Gate
        self.ao = np.zeros((rl+1, ys))
        # Expected Values
        self.eo = np.vstack((np.zeros(eo.shape[0]), eo.T))
        # Define LSTM
        self.LSTM = LSTM(xs, ys, rl, lr)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(x))
    
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def dtangent(self, x):
        return 1 - np.tanh(x) ** 2
        
    # RMS Prop
    def update(self, u):
        self.G = 0.9 * self.G + 0.1 * u ** 2
        self.w -= self.lr/np.sqrt(self.G + 1e-8) * u
        return
    
    def forwardProp(self):
        for i in range(1, self.rl+1):
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            # Store cell state
            self.ca[i] = cs
            self.ha[i] = hs
            self.af[i] = f
            self.ai[i] = inp
            self.ac[i] = c
            self.ao[i] = o
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            self.x = self.eo[i-1]
            
        return self.oa
    
    def backProp(self):
        totalError = 0
        # Cell state
        dfcs = np.zeros(self.ys)
        # Hidden State
        dfhs = np.zeros(self.ys)
        # Weight matrix
        tu = np.zeros((self.ys, self.ys))
        # LSTM Gradients
        # Forget
        tfu = np.zeros((self.ys, self.xs + self.ys))
        # Input
        tiu = np.zeros((self.ys, self.xs + self.ys))
        # Cell
        tcu = np.zeros((self.ys, self.xs + self.ys))
        # Output
        tou = np.zeros((self.ys, self.xs + self.ys))
        # Backwards loop through all states
        for i in range(self.rl, -1, -1):
            error = self.oa[i] - self.eo[i]
            # (error * deriv(h)) * hidden_state 
            tu = np.dot(np.atleast_2d(error * self.dsigmoid(self.oa[i])), np.atleast_2d(self.ha[i]).T)
            # 1. error * RNN weight matrix
            error = np.dot(error, self.w)
            # 2. set input values of LSTM for recurrence i
            self.LSTM.x = np.hstack((self.ha[-1], self.ia[i]))
            # 3. set cell state for recurrence i
            self.LSTM.cs = self.ca[i]
            fu, iu, cu, ou, dfcs, dfhs = self.LSTM.backProp(error, self.ca[i-1], self.af[i], self.ai[i], self.ac[i], self.ao[i], dfcs, dfhs)
            totalError += np.sum(error)
            # Accumulate gradient updates
            tfu += fu
            tiu += iu
            tcu += cu
            tou += ou
            
        # Update LSTM matrices with average of total gradient update
        self.LSTM.update(tfu/self.rl, tiu/self.rl, tcu/self.rl, tou/self.rl)
        # Update Weight matrix with average
        self.update(tu/self.rl)
            
        return totalError
    
    def sample(self):
         #loop through recurrences - start at 1 so the 0th entry of all arrays will be an array of 0's
        for i in range(1, self.rl+1):
            #set input for LSTM cell, combination of input (previous output) and previous hidden state
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            #run forward prop on the LSTM cell, retrieve cell state and hidden state
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            #store input as vector
            maxI = np.argmax(self.x)
            self.x = np.zeros_like(self.x)
            self.x[maxI] = 1
            self.ia[i] = self.x #Use np.argmax?
            #store cell states
            self.ca[i] = cs
            #store hidden state
            self.ha[i] = hs
            #forget gate
            self.af[i] = f
            #input gate
            self.ai[i] = inp
            #cell state
            self.ac[i] = c
            #output gate
            self.ao[i] = o
            #calculate output by multiplying hidden state with weight matrix
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            #compute new input
            maxI = np.argmax(self.oa[i])
            newX = np.zeros_like(self.x)
            newX[maxI] = 1
            self.x = newX
        #return all outputs    
        return self.oa

In [43]:
def LoadText():
    #open text and return input and output data (series of words)
    with open("sherlock.txt", "r") as text_file:
        data = text_file.read()
    # Shorten data
    data = data[0:10000]
    text = list(data)
    outputSize = len(text)
    data = list(set(text))
    uniqueWords, dataSize = len(data), len(data) 
    returnData = np.zeros((uniqueWords, dataSize))
    for i in range(0, dataSize):
        returnData[i][i] = 1
    returnData = np.append(returnData, np.atleast_2d(data), axis=0)
    output = np.zeros((uniqueWords, outputSize))
    for i in range(0, outputSize):
        index = np.where(np.asarray(data) == text[i])
        output[:,i] = returnData[0:-1,index[0]].astype(float).ravel()  
    return returnData, uniqueWords, output, outputSize, data

#write the predicted output (series of words) to disk
def ExportText(output, data):
    finalOutput = np.zeros_like(output)
    prob = np.zeros_like(output[0])
    outputText = ""
    print(len(data))
    print(output.shape[0])
    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            prob[j] = output[i][j] / np.sum(output[i])
        outputText += np.random.choice(data, p=prob)    
    with open("output.txt", "w") as text_file:
        text_file.write(outputText)
    return

In [44]:
#Begin program    
print("Beginning")
iterations = 5000
learningRate = 0.001
#load input output data (words)
returnData, numCategories, expectedOutput, outputSize, data = LoadText()
print("Done Reading")
#init our RNN using our hyperparams and dataset
RNN = RecurrentNeuralNetwork(numCategories, numCategories, outputSize, expectedOutput, learningRate)

#training time!
for i in range(1, iterations):
    #compute predicted next word
    RNN.forwardProp()
    #update all our weights using our error
    error = RNN.backProp()
    #once our error/loss is small enough
    print("Error on iteration ", i, ": ", error)
    if error > -100 and error < 100 or i % 100 == 0:
        #we can finally define a seed word
        seed = np.zeros_like(RNN.x)
        maxI = np.argmax(np.random.random(RNN.x.shape))
        seed[maxI] = 1
        RNN.x = seed  
        #and predict some new text!
        output = RNN.sample()
        print(output)    
        #write it all to disk
        ExportText(output, data)
        print("Done Writing")
print("Complete")

Beginning
Done Reading
Error on iteration  1 :  5400883.94903
Error on iteration  2 :  5389525.22112
Error on iteration  3 :  5381871.36383
Error on iteration  4 :  5375205.61648
Error on iteration  5 :  5369165.83889
Error on iteration  6 :  5363535.46451
Error on iteration  7 :  5358192.12475
Error on iteration  8 :  5353058.88138
Error on iteration  9 :  5348083.73848
Error on iteration  10 :  5343229.67298
Error on iteration  11 :  5338469.25897
Error on iteration  12 :  5333781.53951
Error on iteration  13 :  5329150.0954
Error on iteration  14 :  5324561.79537
Error on iteration  15 :  5320005.95522
Error on iteration  16 :  5315473.75357
Error on iteration  17 :  5310957.81465
Error on iteration  18 :  5306451.90312
Error on iteration  19 :  5301950.69634
Error on iteration  20 :  5297449.61119
Error on iteration  21 :  5292944.6704
Error on iteration  22 :  5288432.39776
Error on iteration  23 :  5283909.73496
Error on iteration  24 :  5279373.97482
Error on iteration  25 :  52

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in less


Done Writing
Error on iteration  101 :  4809675.68426
Error on iteration  102 :  4801170.01424
Error on iteration  103 :  4792627.16817
Error on iteration  104 :  4783999.41817
Error on iteration  105 :  4775285.17479
Error on iteration  106 :  4766482.9801
Error on iteration  107 :  4757591.41247
Error on iteration  108 :  4748609.05451
Error on iteration  109 :  4739534.4791
Error on iteration  110 :  4730366.24217
Error on iteration  111 :  4721102.87842
Error on iteration  112 :  4711742.8985
Error on iteration  113 :  4702284.78693
Error on iteration  114 :  4692727.00051
Error on iteration  115 :  4683067.96694
Error on iteration  116 :  4673306.08362
Error on iteration  117 :  4663439.71654
Error on iteration  118 :  4653467.19925
Error on iteration  119 :  4643386.83179
Error on iteration  120 :  4633196.87975
Error on iteration  121 :  4622895.57323
Error on iteration  122 :  4612481.10581
Error on iteration  123 :  4601951.63356
Error on iteration  124 :  4591305.27401
Error 

KeyboardInterrupt: 